<a href="https://colab.research.google.com/github/HilbertN/Redes_Neuronales/blob/main/RN5_Cometml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
%pip install comet_ml

In [40]:
import comet_ml
comet_ml.init(project_name="Prueba comet")

In [41]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint

In [42]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/hilbertn/prueba-comet/ef51a0b43c484d669ec9060ac1ff55cf



In [43]:
dataset=mnist.load_data()
(x_train, y_train), (x_test, y_test) = dataset

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255  # x_trainv = x_trainv/255
x_test /= 255

num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [44]:
parameters = {
    "batch_size": 128,
    "epochs": 20,
    "optimizer": "adam",
    "loss": "categorical_crossentropy",
}

experiment.log_parameters(parameters)

In [45]:
model = Sequential()
model.add(Input(shape=(28,28))) #Flaten no tiene la opcion input_shape por lo tanto se tiene que agregar esta capa
model.add(Flatten()) #Otra forma de aplanar las imagenes
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))) #Regularizacion L1L2
model.add(Dropout(0.2)) #Fraccion de enlaces a eliminar
model.add(Dense(200)) #Capa lineal , transformacion lineal sin funcion de activacion
model.add(Activation('tanh')) #Se puede agregar despues la funcion de activacion
model.add(Dense(400, activation='selu', kernel_regularizer=regularizers.L1(0.01) )) #Regularizacion L1
model.add(Dense(200, activation='elu', kernel_regularizer=regularizers.L2(l2=1e-4)) ) #Regularizacion L2
model.add(Dense(50,activation='exponential'))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 dense_28 (Dense)            (None, 512)               401920    
                                                                 
 dense_29 (Dense)            (None, 512)               262656    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_30 (Dense)            (None, 200)               102600    
                                                                 
 activation_4 (Activation)   (None, 200)               0         
                                                                 
 dense_31 (Dense)            (None, 400)              

In [ ]:
#El checkpoint es un tipo de callback
# specify the path where you want to save the model
filepath = "best_model.hdf5"

# initialize the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
model.compile(loss=parameters['loss'],optimizer=parameters['optimizer'],metrics=['accuracy'])
model.fit(x_train, y_trainc,
                    batch_size=parameters['batch_size'],
                    epochs=parameters["epochs"],
                    verbose=1,
                    validation_data=(x_test, y_testc),
                    callbacks=[checkpoint])

In [46]:
#Puedes usar comet sin algún callback
model.compile(loss=parameters['loss'],optimizer=parameters['optimizer'],metrics=['accuracy'])
model.fit(x_train, y_trainc,
                    batch_size=parameters['batch_size'],
                    epochs=parameters["epochs"],
                    verbose=1,
                    validation_data=(x_test, y_testc))

Epoch 1/20
469/469 [==============================] - 18s 35ms/step - loss: 6.3314 - accuracy: 0.7590 - val_loss: 0.6553 - val_accuracy: 0.9197
Epoch 2/20
469/469 [==============================] - 16s 34ms/step - loss: 0.5986 - accuracy: 0.9237 - val_loss: 0.4775 - val_accuracy: 0.9468
Epoch 3/20
469/469 [==============================] - 16s 34ms/step - loss: 0.4600 - accuracy: 0.9456 - val_loss: 0.3939 - val_accuracy: 0.9608
Epoch 4/20
469/469 [==============================] - 16s 33ms/step - loss: 0.3948 - accuracy: 0.9586 - val_loss: 0.3777 - val_accuracy: 0.9593
Epoch 5/20
469/469 [==============================] - 15s 32ms/step - loss: 0.3480 - accuracy: 0.9668 - val_loss: 0.3311 - val_accuracy: 0.9682
Epoch 6/20
469/469 [==============================] - 16s 34ms/step - loss: 0.3103 - accuracy: 0.9721 - val_loss: 0.3090 - val_accuracy: 0.9725
Epoch 7/20
469/469 [==============================] - 16s 33ms/step - loss: 0.2877 - accuracy: 0.9752 - val_loss: 0.2866 - val_accuracy:

In [48]:
#Guarda el modelo, este es del código donde usamos el callback checkpoint
experiment.log_model("MNIST1", "best_model.hdf5")

In [49]:
#Esto registrará el código del cuaderno y se asegurará de que todos los datos se hayan enviado a Comet.
experiment.end()